In [1]:
import cv2
import numpy as np
import os
import pickle
import csv
from datetime import datetime
from mtcnn import MTCNN
import face_recognition

# Create necessary folders
os.makedirs("data", exist_ok=True)
os.makedirs("Attendance", exist_ok=True)

# Initialize face detector
detector = MTCNN()


In [2]:
# 2. Function to Add Faces
def add_face(name, max_samples=100):
    if not name:
        print("Please provide a valid name.")
        return

    cap = cv2.VideoCapture(0)
    encodings_list = []
    count = 0

    print("🟡 Starting face collection... Press 'q' to stop early.")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        detections = detector.detect_faces(rgb)

        for det in detections:
            x, y, w, h = det['box']
            top, right, bottom, left = y, x + w, y + h, x
            face_location = [(top, right, bottom, left)]

            enc = face_recognition.face_encodings(rgb, face_location)
            if enc:
                encodings_list.append(enc[0])
                count += 1

                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(frame, f"Collected: {count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

        cv2.imshow("Add Face", frame)
        if cv2.waitKey(1) & 0xFF == ord('q') or count >= max_samples:
            break

    cap.release()
    cv2.destroyAllWindows()

    # Save encodings
    if encodings_list:
        if os.path.exists("data/face_encodings.pkl"):
            with open("data/face_encodings.pkl", "rb") as f:
                all_encodings = pickle.load(f)
        else:
            all_encodings = []

        all_encodings.extend(encodings_list)
        with open("data/face_encodings.pkl", "wb") as f:
            pickle.dump(all_encodings, f)

        # Save names
        new_names = [name] * len(encodings_list)
        if os.path.exists("data/names.pkl"):
            with open("data/names.pkl", "rb") as f:
                all_names = pickle.load(f)
        else:
            all_names = []

        all_names.extend(new_names)
        with open("data/names.pkl", "wb") as f:
            pickle.dump(all_names, f)

        print(f"✅ {len(encodings_list)} faces added for '{name}'.")
    else:
        print("❌ No faces detected.")


In [ ]:
add_face("Shivam")


🟡 Starting face collection... Press 'q' to stop early.


In [11]:
def mark_attendance(threshold=0.5):
    if not os.path.exists("data/face_encodings.pkl"):
        print("❌ No face data found. Please add faces first.")
        return

    with open("data/face_encodings.pkl", "rb") as f:
        known_encodings = pickle.load(f)
    with open("data/names.pkl", "rb") as f:
        known_names = pickle.load(f)

    cap = cv2.VideoCapture(0)
    present = set()
    print("🔵 Marking attendance... Press 'q' to stop.")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        detections = detector.detect_faces(rgb)

        for det in detections:
            x, y, w, h = det['box']
            face_location = [(y, x + w, y + h, x)]
            enc = face_recognition.face_encodings(rgb, face_location)
            if not enc:
                continue

            encoding = enc[0]
            distances = face_recognition.face_distance(known_encodings, encoding)
            min_dist = np.min(distances)
            idx = np.argmin(distances)
            name = known_names[idx] if min_dist < threshold else "Unknown"

            if name != "Unknown" and name not in present:
                present.add(name)
                now = datetime.now()
                date = now.strftime("%d-%m-%Y")
                time_str = now.strftime("%H:%M:%S")
                filename = f"Attendance/Attendance_{date}.csv"
                file_exists = os.path.isfile(filename)

                with open(filename, "a", newline='') as f:
                    writer = csv.writer(f)
                    if not file_exists:
                        writer.writerow(["NAME", "DATE", "TIME"])
                    writer.writerow([name, date, time_str])
                print(f"✅ Marked: {name}")

            cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)
            cv2.putText(frame, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

        cv2.imshow("Mark Attendance", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    print(f"🟢 Attendance done. Present: {present}")


In [12]:
#calling

mark_attendance(threshold=0.5)




🔵 Marking attendance... Press 'q' to stop.
✅ Marked: Harsh
✅ Marked: Aakash
✅ Marked: Shruti
🟢 Attendance done. Present: {'Aakash', 'Harsh', 'Shruti'}
